<a href="https://colab.research.google.com/github/fwitschel/QDMKM/blob/main/notebooks/indexing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

# Execute this code only if in colab
if 'COLAB_GPU' in os.environ:
  print("Executing in Colab!")
  # Cloning GitHub repository
  !git clone https://github.com/fwitschel/QDMKM.git
  %cd QDMKM


Executing in Colab!
Cloning into 'QDMKM'...
remote: Enumerating objects: 33, done.
remote: Counting objects: 100% (33/33), done.
remote: Compressing objects: 100% (22/22), done.
remote: Total 33 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (33/33), 20.91 KiB | 4.18 MiB/s, done.
Resolving deltas: 100% (3/3), done.
/content/QDMKM


In [2]:
!pip install langchain langchain-community rank_bm25 pypdf unstructured chromadb
!pip install unstructured['pdf'] unstructured
!apt-get install poppler-utils
!apt-get install -y tesseract-ocr
!apt-get install -y libtesseract-dev
!pip install pytesseract
!pip install bitsandbytes accelerate peft safetensors sentencepiece
!pip install nltk
!pip install -U sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 35 not upgraded.
Need to get 186 kB of archives.
After this operation, 697 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.10 [186 kB]
Fetched 186 kB in 1s (215 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 126374 files and directories currently installed.)
Preparing to unpack .../poppler-utils_22.02.0-2ubuntu0.10_amd64.deb ...
Unpacking poppler-utils (22.02.0-2ubuntu0.10) ...
Setting up poppler-utils (22.02.0-2ubuntu0.10) ...
Processing triggers for man-db (2.10.2-1) ...
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly insta

In [3]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma

from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain.llms import HuggingFaceHub
import torch
from transformers import ( AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline, )
from langchain import HuggingFacePipeline
from sentence_transformers import SentenceTransformer

from langchain.retrievers import BM25Retriever, EnsembleRetriever

import os
import nltk
import pandas as pd
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
emails = pd.read_csv("/content/QDMKM/data/cases-emails.csv")
print(emails.head())

               sender                 receiver  \
0   student42@MBFH.ch      my_msc_dean@MBFH.ch   
1   student21@MBFH.ch      my_msc_dean@MBFH.ch   
2   student84@MBFH.ch  former_msc_dean@MBFH.ch   
3  student168@MBFH.ch      my_msc_dean@MBFH.ch   
4  student336@MBFH.ch      my_msc_dean@MBFH.ch   

                                    subject  first_date notice_weeks  \
0                       Re: late submission  2019-06-16            1   
1  Re: extend deadline because of sickness?  2020-06-14            1   
2                     Re: thesis submission  2016-07-01          NaN   
3                 Re: request for extension  2021-05-10            6   
4                             Re: extension  2019-05-16            5   

                   tags                                           all_text  \
0  sickness certificate  Dear Ms Smith, unfortunately, we cannot accept...   
1  sickness certificate  Dear Mr Doe, your request to extend the deadli...   
2                   job  Dear Ms

In [ ]:
from langchain.document_loaders import PyPDFLoader
doc_path = "/content/drive/MyDrive/rueckholung.pdf"
loader = PyPDFLoader(doc_path)
pages = loader.load_and_split()

In [ ]:
import nltk
from nltk.stem.snowball import SnowballStemmer

nltk.download('stopwords')

def preprocess(text):

    # Tokenize the text
    tokens = nltk.word_tokenize(text)

    # lowercase the tokens
    tokens = [token.lower() for token in tokens]

    # stemming
    stemmer = SnowballStemmer('german', ignore_stopwords=True)
    tokens = [stemmer.stem(token) for token in tokens]

    return tokens

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from langchain_core.documents import Document

#text = pages[12].page_content
text1 = "Wandern im Schweizer Jura. Die schönsten Routen und besten Einkehrmöglichkeiten"
text2 = "So verschönern Sie Ihre Wände: Wandschmuck und die neusten Tapetenmuster"

preprocessed_text1 = preprocess(text1)
print(preprocessed_text1)

preprocessed_text2 = preprocess(text2)
print(preprocessed_text2)

keyword_retriever = BM25Retriever.from_documents(pages, preprocess_func=preprocess, k=5)

['wand', 'im', 'schweiz', 'jura', '.', 'die', 'schon', 'rout', 'und', 'best', 'einkehrmog']
['so', 'verschon', 'sie', 'ihre', 'wand', ':', 'wandschmuck', 'und', 'die', 'neust', 'tapetenmust']
